In [1]:
import flickrapi
from flickrapi import FlickrAPI
from bs4 import BeautifulSoup
import xml
from urllib.request import urlopen
from urllib.request import urlretrieve
import csv
import pandas as pd
import numpy as np
import time
import os
import logging
import re
from tqdm import tqdm


logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# create a file handler
handler = logging.FileHandler('hello.log')
handler.setLevel(logging.INFO)
logger.addHandler(handler)


FLICKR_PUBLIC = '90cb7b5c1ea80af5263116dd84219cbc'
FLICKR_SECRET = '55321131ef96714b'
flickr = FlickrAPI(FLICKR_PUBLIC, FLICKR_SECRET, format='parsed-json')

df = pd.read_csv('/mnt/metadata.csv')

In [2]:
df['image_id'] = df['image_id'].astype(str)

In [3]:
done_lines = os.listdir('/mnt/images/')
done_lines = [re.sub('.jpg', '', x) for x in done_lines]
pending_lines = list(set(df['image_id'].tolist()) - set(done_lines))

In [5]:
len(pending_lines)

15112

In [ ]:
for row in tqdm(pending_lines):
    try:
        image_id = row.strip()
        file_location = '/mnt/images/%s.jpg'%(image_id)
        image = flickr.photos.getinfo(photo_id = image_id)
        secret = image['photo']['secret']
        server = image['photo']['server']
        farm_id = image['photo']['farm']
        urlretrieve('https://farm%s.staticflickr.com/%s/%s_%s.jpg'%(farm_id, server, image_id, secret), file_location)
        time.sleep(0.2)
    except:
        logging.error('error while processing %s'%(image_id))

  1%|          | 130/15112 [01:39<3:11:58,  1.30it/s]